# Math AI Playground

- Evaluate numeric-only math expression step by step.
- Expression length cannot be more than 20 characters.
- Support `*` and `+` only, `-` and `()` support will be attempted later. Intended for the neural network to learn the rules of mathematics.
- Support integers only.

Character encoding:

| Character | Token    |
|-----------|----------|
| 0         | 0        |
| 1         | 1        |
| 2         | 2        |
| 3         | 3        |
| 4         | 4        |
| 5         | 5        |
| 6         | 6        |
| 7         | 7        |
| 8         | 8        |
| 9         | 9        |
| *         | 10       |
| +         | 11       |
| -         | 12       |
|  (space)  | 13       |

In [21]:
# Global imports

from typing import List, Tuple

import random
import re

import pandas as pd

import tensorflow as tf
from tensorflow import keras
from keras import layers
import numpy as np


randomer = random.Random(1)

In [22]:
CHAR_TOKEN_MAP = {
    '0': 0,
    '1': 1,
    '2': 2,
    '3': 3,
    '4': 4,
    '5': 5,
    '6': 6,
    '7': 7,
    '8': 8,
    '9': 9,
    '*': 10,
    '+': 11,
    '-': 12,
    ' ': 13
}

TOKEN_CHAR_MAP = {}

for k, v in CHAR_TOKEN_MAP.items():
    TOKEN_CHAR_MAP[v] = k

TOKEN_CHAR_MAP

{0: '0',
 1: '1',
 2: '2',
 3: '3',
 4: '4',
 5: '5',
 6: '6',
 7: '7',
 8: '8',
 9: '9',
 10: '*',
 11: '+',
 12: '-',
 13: ' '}

In [28]:
CHAR_VOCAB = list(CHAR_TOKEN_MAP)
CHAR_VOCAB

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '*', '+', '-', ' ']

## Data generation

In [24]:
DIGIT_CHARS = list('0123456789')
OP_CHARS = list('*+')
EXPRESSION_LENGTH = 20
TOKEN_SPACE_SIZE = 16

def generate_initial_expression(rand: random.Random) -> str:
    initial_expression = [rand.choice(DIGIT_CHARS) if i % 3 < 2 else rand.choice(OP_CHARS) for i in range(14)]
    if initial_expression[0] == '0':
        initial_expression = initial_expression[1:]
    initial_expression = re.sub("([\*\+\-])0", lambda mo: mo.group(1), ''.join(initial_expression))
    
    return initial_expression

# Evaluate one step of the expression, return expression, False if no further
# steps can be made anymore.
def progress_expression_step(expression: str) -> Tuple[str, bool]:
    op = ''
    op_idx = -1

    if '*' in expression:
        # Multiplication takes precendence
        op = '*'
        op_idx = expression.find('*')
    else:
        m = re.search('\+', expression)
        if m is not None:
            op = expression[m.start()]
            op_idx = m.start()
    
    if op == ' ' or op_idx == -1:
        return expression, False
    
    start_idx = op_idx - 1
    end_idx = op_idx + 1

    while start_idx - 1 >= 0 and expression[start_idx - 1] not in OP_CHARS:
        start_idx -= 1
    
    while end_idx + 1 < len(expression) and expression[end_idx + 1] not in OP_CHARS:
        end_idx += 1

    num1 = int(expression[start_idx:op_idx])
    num2 = int(expression[op_idx+1:end_idx+1])

    calc_result = 0

    if op == '*':
        calc_result = num1 * num2
    elif op == '+':
        calc_result = num1 + num2
    elif op == '-':
        calc_result = num1 - num2

    before_result = expression[:start_idx]
    after_result = expression[end_idx+1:]

    if len(before_result) > 0 and calc_result < 0:
        if before_result[-1] == '-':
            before_result[-1] = '+'
        elif before_result[-1] == '+':
            before_result[-1] = '-'
    
    calc_result = abs(calc_result)

    return before_result + str(calc_result) + after_result, True
    

def generate_expression_with_steps(rand: random.Random) -> List[str]:
    initial_expression = generate_initial_expression(rand)
    ret = [initial_expression]
    while True:
        exp, has_further = progress_expression_step(ret[-1])
        if not has_further:
            break
        ret.append(exp)
    if int(eval(initial_expression)) != int(ret[-1]):
        raise ValueError("internal logic error, value evaluation is incorrect")
    return [l.ljust(20) for l in ret]

generate_expression_with_steps(randomer)

['29*41+77+31+6       ',
 '1189+77+31+6        ',
 '1266+31+6           ',
 '1297+6              ',
 '1303                ']

In [27]:
randomer = random.Random(1)

from_expression_train_file = open('data/from_expression_train.txt', "w")
to_expression_train_file = open('data/to_expression_train.txt', "w")

for i in range(10**6):
    steps = generate_expression_with_steps(randomer)
    for j in range(len(steps)-1):
        from_expression_train_file.write(steps[j] + "\n")
        to_expression_train_file.write(steps[j+1] + "\n")

from_expression_train_file.close()
to_expression_train_file.close()

from_expression_test_file = open('data/from_expression_test.txt', "w")
to_expression_test_file = open('data/to_expression_test.txt', "w")

for i in range(2 * 10**5):
    steps = generate_expression_with_steps(randomer)
    for j in range(len(steps)-1):
        from_expression_test_file.write(steps[j] + "\n")
        to_expression_test_file.write(steps[j+1] + "\n")

from_expression_test_file.close()
to_expression_test_file.close()

from_expression_validation_file = open('data/from_expression_validation.txt', "w")
to_expression_validation_file = open('data/to_expression_validation.txt', "w")

for i in range(2 * 10**5):
    steps = generate_expression_with_steps(randomer)
    for j in range(len(steps)-1):
        from_expression_validation_file.write(steps[j] + "\n")
        to_expression_validation_file.write(steps[j+1] + "\n")

from_expression_validation_file.close()
to_expression_validation_file.close()